## Frameworks de alto nivel (PyTorch Lightning)

<a target="_blank" href="https://github.com/pglez82/DeepLearningWeb/blob/master/labs/notebooks/Frameworks%20alto%20nivel%20(PyTorch%20Lightning).ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>